<a href="https://colab.research.google.com/github/chanos17/FLOWER/blob/main/FLOWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import cv2
from sklearn.model_selection import train_test_split


In [19]:
import os
import re
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# 데이터셋 클래스 정의
class CellDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(data_folder) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.data_folder, img_name)
        image = Image.open(img_path).convert("RGB")

        # 이미지 이름에서 레이블 추출
        label = int(re.search(r'box_(\d+)', img_name).group(1))

        if self.transform:
            image = self.transform(image)

        return image, label

# 데이터셋 경로 및 변환 정의
data_folder = '/content/Data_Folder'
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# 데이터셋 및 데이터로더 생성
dataset = CellDataset(data_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


In [20]:
import torch.nn as nn
import torch.nn.functional as F

class CellCNN(nn.Module):
    def __init__(self):
        super(CellCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.conv3 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64*14*14, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [21]:
model = CellCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [22]:
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        # 레이블을 float 형태로 변환
        labels = labels.float()

        # 옵티마이저 초기화
        optimizer.zero_grad()

        # 모델 예측
        outputs = model(images)

        # 손실 계산
        loss = criterion(outputs.squeeze(), labels)

        # 역전파 및 옵티마이저 스텝
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')


Epoch [1/50], Loss: 112.0326
Epoch [2/50], Loss: 100.1496
Epoch [3/50], Loss: 80.6561
Epoch [4/50], Loss: 52.8079
Epoch [5/50], Loss: 22.6792
Epoch [6/50], Loss: 16.0271
Epoch [7/50], Loss: 40.8453
Epoch [8/50], Loss: 29.3941
Epoch [9/50], Loss: 15.4197
Epoch [10/50], Loss: 12.8045
Epoch [11/50], Loss: 16.6104
Epoch [12/50], Loss: 20.7613
Epoch [13/50], Loss: 22.6982
Epoch [14/50], Loss: 22.1257
Epoch [15/50], Loss: 19.6666
Epoch [16/50], Loss: 16.3446
Epoch [17/50], Loss: 13.5256
Epoch [18/50], Loss: 12.5835
Epoch [19/50], Loss: 13.9806
Epoch [20/50], Loss: 16.3081
Epoch [21/50], Loss: 17.1820
Epoch [22/50], Loss: 15.8733
Epoch [23/50], Loss: 13.8251
Epoch [24/50], Loss: 12.6501
Epoch [25/50], Loss: 12.7622
Epoch [26/50], Loss: 13.6087
Epoch [27/50], Loss: 14.4317
Epoch [28/50], Loss: 14.7383
Epoch [29/50], Loss: 14.4107
Epoch [30/50], Loss: 13.6550
Epoch [31/50], Loss: 12.8868
Epoch [32/50], Loss: 12.5450
Epoch [33/50], Loss: 12.8185
Epoch [34/50], Loss: 13.3437
Epoch [35/50], Loss: 

In [24]:
def predict_cell_count(model, image_path):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # 배치 차원 추가

    with torch.no_grad():
        output = model(image)
        predicted_count = output.item()

    return predicted_count

# 예측하기
image_path = '/content/Data_Folder/Testdata/MS1096_969OE_29_male_left_2_A.png'
predicted_count = predict_cell_count(model, image_path)
print(f'Predicted Cell Count: {predicted_count:.0f}')


Predicted Cell Count: 10


In [25]:
def predict_cell_count(model, image_path):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # 배치 차원 추가

    with torch.no_grad():
        output = model(image)
        predicted_count = output.item()

    return predicted_count

# 예측하기
image_path = '/content/Data_Folder/Testdata/MS1096_969OE_29_male_left_2_B.png'
predicted_count = predict_cell_count(model, image_path)
print(f'Predicted Cell Count: {predicted_count:.0f}')


Predicted Cell Count: 11
